# Recursos
Bathia (2010) <br>
Horta (2018) <br>
Kokoszka (2019)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# data
lqdensities_path = "../data/processed/lqdensities.xlsx"

lqdensities = pd.read_excel(lqdensities_path, index_col="x")
lqdensities.head()

,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,t_9,t_10,...,t_112,t_113,t_114,t_115,t_116,t_117,t_118,t_119,t_120,t_121
x,,,,,,,,,,,,,,,,,,,,,
0.000000,-3.510057,-4.112028,-3.245852,-3.451768,-3.010697,-2.416859,-2.941917,-3.757795,-4.049503,-2.462951,...,-4.106067,-3.044384,-3.499390,-3.806129,-3.256140,-3.627464,-3.130958,-2.843934,-2.740841,-2.971827
0.005025,-3.831881,-4.280989,-3.595798,-3.755390,-3.047621,-1.405972,-2.863023,-4.039823,-4.279095,-2.480686,...,-4.258328,-3.449396,-3.708159,-3.974514,-3.448629,-3.867496,-3.486383,-2.981364,-2.808421,-3.219460
0.010050,-4.056523,-4.415420,-3.900718,-3.986254,-3.194371,-2.904361,-3.497128,-4.238690,-4.443333,-2.980616,...,-4.371172,-3.737058,-3.837260,-4.091067,-3.596603,-4.040252,-3.728953,-2.983511,-2.846394,-3.431278
0.015075,-4.234897,-4.528753,-4.172718,-4.185588,-3.444439,-3.536240,-4.004996,-4.391251,-4.567315,-3.386732,...,-4.458680,-3.952467,-3.922403,-4.180115,-3.821583,-4.184463,-3.907602,-3.041303,-2.974170,-3.652233
0.020101,-4.385658,-4.626079,-4.403389,-4.360551,-3.745242,-3.871029,-4.325151,-4.514555,-4.664294,-3.631419,...,-4.528478,-4.119930,-3.981792,-4.254348,-4.085199,-4.312487,-4.044709,-3.367491,-3.397260,-3.870436


In [3]:
# Defining the inner product on L^2([a,b])
# inner.product = function(f, g, du) {drop(crossprod(f, g))*du}
def inner_product(f, g, du):
    """
    f, g = vectors
    """
    ip = (f.T @ g)*du
    return ip

#The du in this context represents the differential element or step size in numerical integration, 
# specifically for approximating the inner product in the function space L²([a,b]).
# du is the spacing between sample points

In [4]:
# Defining the L2-norm
# L2norm = function(f, du) {sqrt(inner.product(f, f, du))}
def L2norm(f,du):
    norm = np.sqrt(inner_product(f, f, du))
    return norm

In [5]:
# Nxp: row=number_of_daily_observations and p=number_of_curves

#3 curves with dimension 2
matrixX = np.array([[1, 2],
                    [3, 4],
                    [5, 6]])

# 4 curves with dimension 5
Y = np.array([  [2, 1, 3, 4],
                [1, 0, 2, 1],
                [3, 2, 1, 0],
                [1, 2, 3, 4],
                [4, 3, 2, 1]])

In [6]:
 # Interval length (based on equal distance between grid points)
# m = 5001
# du = u[2] - u[1]
du = lqdensities.reset_index()["x"].diff()[1]
du = 0.005
p=5 # script original
p=2 # para poder utilizar matriz pequena
#m = 5001 # number of grid points
m = Y.shape[0]
lag_max = 3

# Y : matrix
#  n = N = ncol(Y)
n = N = Y.shape[1]

In [26]:
m

5

In [7]:
# Defining Ybar
# Ybar = rowMeans(Y)
Ybar = np.mean(Y, axis=1, keepdims=True) # rowMeans (point-wise mean)
Ybar

array([[2.5],
       [1. ],
       [1.5],
       [2.5],
       [2.5]])

In [8]:
# Defining the deviation function Ydev = Y - Ybar, which is used as an input
# by the function inner.product in constructing the matrix Kstar
# Ydev = Y - Ybar

Ydev = Y - Ybar
Ydev

array([[-0.5, -1.5,  0.5,  1.5],
       [ 0. , -1. ,  1. ,  0. ],
       [ 1.5,  0.5, -0.5, -1.5],
       [-1.5, -0.5,  0.5,  1.5],
       [ 1.5,  0.5, -0.5, -1.5]])

In [9]:
    # 3. Creating the matrix 'Kstar'    
    # Building the 'core' matrices of Kstar. Below we define the matrices core (n x n),
    # Kstar.core0 [(n-p) x (n-p)] and the array Kstar.core [(n-p) x (n-p) x p].
    # We have that
    #   Kstar = (Kstar.core[,,1] + ... + Kstar.core[,,p])%*%Kstar.core0
    # Where
    #   Kstar.core0[t,s] = <Y[,t],Y[,s]>, t,s in {1,...,n-p}
    #   Kstar.core[t,s,k] = <Y[,t+k],Y[,s+k]>, t,s in {1,...,n-p} and k in {1,...,p}
    # Thus, the matrix core, defined by
    #   core[t,s] = <Y[,t],Y[,s]>, t,s in {1,...,n}
    # contains all the relevant information regarding Kstar.core0 and Kstar.core,
    # and may be used as a building block for the latter matrices.

    # core = inner.product(Ydev,Ydev, du)
    # Kstar.core0 = core[1:(n-p),1:(n-p)]
    # Kstar.core = array(0,c(n-p,n-p,p))
    # for (k in 1:p) Kstar.core[,,k] = core[(k+1):(n-(p-k)),(k+1):(n-(p-k))]

core = inner_product(Ydev, Ydev, du=0.05)
core

array([[ 0.35 ,  0.15 , -0.125, -0.375],
       [ 0.15 ,  0.2  , -0.125, -0.225],
       [-0.125, -0.125,  0.1  ,  0.15 ],
       [-0.375, -0.225,  0.15 ,  0.45 ]])

In [10]:
Kstar_core0 = core[:(n-p),:(n-p)]
Kstar_core0

array([[0.35, 0.15],
       [0.15, 0.2 ]])

In [11]:
# The first dimension (rows) has size n-p.
# The second dimension (columns) has size n-p.
# The third dimension (slices) has size p

Kstar_core = np.zeros((n-p, n-p, p))
Kstar_core

array([[[0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.]]])

In [12]:
for k in range(p):
    Kstar_core[:, :, k] = core[k : n-(p-k)-1, k : n-(p-k)-1]

In [13]:
Kstar_core

array([[[0.35, 0.2 ],
        [0.35, 0.2 ]],

       [[0.35, 0.2 ],
        [0.35, 0.2 ]]])

In [14]:
Kstar_core = np.zeros((n-p, n-p, p))
Kstar_core

array([[[0., 0.],
        [0., 0.]],

       [[0., 0.],
        [0., 0.]]])

In [15]:
for k in range(p):
    row_index = k+1
    col_index = n-(p-k)+1
    Kstar_core[:, :, k] = core[row_index:col_index , row_index:col_index]

In [16]:
Kstar_core

array([[[ 0.2  ,  0.1  ],
        [-0.125,  0.15 ]],

       [[-0.125,  0.15 ],
        [ 0.1  ,  0.45 ]]])

In [17]:
#print(Kstar_core) não corresponde ao print do R. É necessário printar um de cada vez, já que o numpy faz o display
#de arrays 3D.
for k in range(p):
    print(f"\nSlice {k+1}:")
    print(Kstar_core[:, :, k])


Slice 1:
[[ 0.2   -0.125]
 [-0.125  0.1  ]]

Slice 2:
[[0.1  0.15]
 [0.15 0.45]]


In [18]:
# Summing the matrices in 'Kstar.core'
# Kstar.sum = matrix(0,nrow=n-p,ncol=n-p)
# for (k in 1:p) Kstar.sum = Kstar.sum + Kstar.core[,,k]

Kstar_sum = np.zeros([(n-p),(n-p)]) 
for k in range(p):
    Kstar_sum = Kstar_sum + Kstar_core[:,:,k]
Kstar_sum

array([[0.3  , 0.025],
       [0.025, 0.55 ]])

In [19]:
# Defining Kstar
# Kstar = (n-p)^(-2) * Kstar.sum %*% Kstar.core0
Kstar = ((n-p)**(-2)) * (Kstar_sum.T @ Kstar_core0)
Kstar

array([[0.0271875, 0.0125   ],
       [0.0228125, 0.0284375]])

In [20]:
# 4. Eigen-analisys

# Getting the eigenvalues and eigenvectors from 'Kstar'

# Storing the eigenvalues; length(thetahat)=n-p
# thetahat = eigen(Kstar)$values
# Storing the eigenvectors; each column of gammahat corresponds to
# one eigenvector; dim(gammahat)=[(n-p) x (n-p)]
# gammahat = eigen(Kstar)$vectors
eigenvalues, eigenvectors = np.linalg.eig(Kstar)
idx = eigenvalues.argsort()[::-1] 
thetahat = eigenvalues[idx]
gammahat = eigenvectors[:,idx]

In [21]:
# Defining a tolerance level for 'testing' if the imaginary part of the
# eigenvalues and eigenvectors is zero
tol = 10**(-4)

# Checking if there are any complex eigenvalues (among the first eleven)
# for (j in 1:10){
#     if (abs(Im(thetahat[j]))>tol) print("Complex eigenvalue found.")
# }

for j in range(np.min([len(thetahat),10])):
    if (abs(thetahat[j].imag)>tol):
        print("Complex eigenvalue found.")

thetahat = thetahat.real
gammahat = gammahat.real # diferença para código original, apra não ter que aplicar Re de novo na parte de bootstrap

# Parte abaixo serve apenas para ordenar autovalores e autovetores conjuntamente do maior para o menor, o que já foi feito
# thetahat = sort(thetahat, index.return=TRUE, decreasing=TRUE)
# thetahat.index = thetahat$ix
# thetahat = thetahat$x
# gammahat.temp = matrix(0, nrow=nrow(gammahat), ncol=ncol(gammahat))
# for(j in 1:(length(thetahat)))
# {
#   gammahat.temp[,j] = gammahat[,thetahat.index[j]]
# }
# gammahat = gammahat.temp

In [22]:
# # Storing the original eigenvalues and eigenvectors
# thetahat.old = thetahat
# gammahat.old = gammahat
thetahat_old = thetahat
gammahat_old = gammahat

In [33]:
# Selecting the number of components
# if(select_ncomp == "TRUE")
select_ncomp = True
lag_max = 2
if select_ncomp == True:
    bs_pvalues = np.full(lag_max, 0)
    # Building the estimated functions Yhat with dimension d0, d0 in {1,2,...,lag_max}.
    # See the main code for an explanation of this section.
    # for(d0 in 1:lag_max)
    # {
    # thetahatH0 = Re(thetahat.old[d0+1])
    # thetahat = Re(thetahat.old[1:d0])
    # gammahat = Re(gammahat.old[,1:d0])
    # psihat.root = Ydev[,1:(n-p)] %*% gammahat
    # psihat = matrix(0, nrow = m, ncol = d0)
    # for(i in 1:d0) psihat[,i] = psihat.root[,i]/L2norm(psihat.root[,i], du = du); rm(i)
    for d0 in range(lag_max-1): # adicionado -1 para slicing não ultrapassar limites. Ver teste de hipótese do artigo (lambda_{d0+1}=0)
        # print(d0)
        thetahatH0 = thetahat_old[d0+1]
        thetahat   = thetahat_old[d0]
        gammahat   = gammahat_old[:,:(d0+1)]
        psihat_root = Ydev[:,:(n-p)] @ gammahat_old[:,:(d0+1)]
        psihat = np.zeros([m, d0+1])
        # for(i in 1:d0) psihat[,i] = psihat.root[,i]/L2norm(psihat.root[,i], du = du); rm(i)
        for i in range(d0+1):
            print(i)
            psihat[ : , i ] = psihat_root[:,i]/L2norm(psihat_root[:,i], du=du)
            # del i
        # etahat = inner.product(psihat, Ydev, du = du)
        etahat = inner_product(psihat, Ydev, du=du)
        # Yhat = Ybar + psihat %*% etahat
        Yhat = Ybar + (psihat @ etahat) 

0


In [28]:
Ybar

array([[2.5],
       [1. ],
       [1.5],
       [2.5],
       [2.5]])

In [27]:
psihat

array([[ 7.63020796],
       [ 4.1096168 ],
       [-6.45215665],
       [ 6.45215665],
       [-6.45215665]])

In [30]:
etahat = inner_product(psihat, Ydev, du=du)
etahat

array([[-0.16424904, -0.12616582,  0.08801478,  0.20240008]])

In [32]:
psihat @ etahat

array([[-1.25325437, -0.96267143,  0.67157107,  1.54435473],
       [-0.67500063, -0.51849317,  0.36170701,  0.83178679],
       [ 1.05976056,  0.81404163, -0.56788514, -1.30591705],
       [-1.05976056, -0.81404163,  0.56788514,  1.30591705],
       [ 1.05976056,  0.81404163, -0.56788514, -1.30591705]])